In [35]:
import warnings
warnings.filterwarnings('ignore')
import sys, glob
sys.path.append('/code')
from tools.docking.vina import vina_dock_multi
from tools.docking.preparation import define_vina_box
from example_notebooks.ui.file_widgets import PathSelector, DockButtons, InputBoxSet, log_progress
import os, shutil
from tqdm import tqdm

# Docking (vina) multiple ligands in a box defined by a reference ligand
---

### 1. Selecting some inputs:

---

**First we need to select some inputs, namely:**
- the protein we want to dock into (pdb file)
- the reference ligand to define a box (mol/sdf)
- the ligands we want to dock (sdf)
- tan output directory

**To do this:**  
*(N.B. run the next 2 cells to show the file dialogues)*
1. click on the arrow in the dropdown box at the top
2. select the relevant file in the dialogue 
3. click the button next to either protein, query ligands or reference to set the file

*repeat this until all files are selected*

4. type the directory you want to get the results of the docking in into the 'Docking dir' input
5. select 'Set' to set the directory if the directory exists or create to create it, and then set to set it



In [36]:
f = PathSelector(start_dir=os.path.join(os.getcwd()))
display(f.accord)
b = DockButtons(f)
b.render_buttons()

A Jupyter Widget

A Jupyter Widget

In [37]:
i = InputBoxSet(tlabel='Docking dir:')
i.render()

A Jupyter Widget

Created: /data/docking/XX02KALRNA-x1389_1
/data/docking/XX02KALRNA-x1389_1


---

### 2. Check the inputs 
---

Run the next cell, and check you are happy with the inputs and docking directory


In [38]:
pdb_file = b.pdb_button.description
ligands_file = b.lig_button.description
ref_file = b.ref_button.description
conf_file = 'conf.txt'
docking_directory = i.inbox.value

print(f'inputs: \n\t - pdb: {pdb_file} \n\t - ligs: {ligands_file} \
      \n\t - ref: {ref_file} \n\ndocking directory: {docking_directory}')

inputs: 
	 - pdb: /data/XX02KALRNA-x1389_1/XX02KALRNA-x1389_1.pdb 
	 - ligs: /data/XX02KALRNA-x1389_1/250_similar_sucos_diverse.sdf       
	 - ref: /data/XX02KALRNA-x1389_1/XX02KALRNA-x1389_1.sdf 

docking directory: /data/docking/XX02KALRNA-x1389_1


---

### 3. Define a box from the reference ligand
---

Define a box around your reference ligand, and then write it out to a configuration file for autodock vina

There are some parameters that you can change here:

```
define_vina_box:
    Use a reference ligand to produce a box definition for autodock vina
    
        :param ref_lig: a mol or sdf file of the reference ligand
        :param output: the name of the output file
        :param bufx: distance (A) of buffer in x direction (default=10)
        :param bufy: distance (A) of buffer in y direction (default=10)
        :param bufz: distance (A) of buffer in z direction (default=10)
        :param exhaustiveness: exhaustivness of the vina calculation (default=20)
        :param seed: random seed for vina (default=None)
        :param cpu: number of cpu's to use in vina calculation
```

In [39]:
define_vina_box(ligands_file, conf_file)

---

### 4. Dock!
---

There are some parameters that you can change here:

```
vina_dock_multi:
    Dock multiple ligands from an sdf file against a protein, with a given configuration file with vina. 
    Also does preparation from pdb and sdf
    
        :param protein_pdb: the input pdb file to prepare and dock against
        :param ligands_sdf: an sdf file of multiple ligands to split, prepare and dock
        :param conf: the path of a configuration file (required for now)
        :param docking_dir: the output directory
        :param multiproc: use multi-processing? (boolean)
        :param exe: the path to the vina executable
```

But if you have followed the steps in the above boxes, you can just run this cell without a worry!


In [40]:
vina_dock_multi(protein_pdb=pdb_file, ligands_sdf=ligands_file, 
                conf=conf_file, docking_dir=docking_directory, multiproc=True)

docking_dir=docking_directory

if not os.path.isdir(os.path.join(docking_dir, 'docked/')):
    os.mkdir(os.path.join(docking_dir, 'docked/'))
if not os.path.isdir(os.path.join(docking_dir, 'logs/')):
    os.mkdir(os.path.join(docking_dir, 'logs/'))
if not os.path.isdir(os.path.join(docking_dir, 'inputs/')):
    os.mkdir(os.path.join(docking_dir, 'inputs/'))

docked_files = glob.glob(os.path.join(docking_dir, '*_docked.pdbqt'))
for f in docked_files:
    shutil.move(f, os.path.join(docking_dir, 'docked/' + f.split('/')[-1]))

logs = glob.glob(os.path.join(docking_dir, '*_docked.log'))
for f in logs:
    shutil.move(f, os.path.join(docking_dir, 'logs/' + f.split('/')[-1]))

in_ligs = glob.glob(os.path.join(docking_dir, '*_prepared.pdbqt'))
for f in in_ligs:
    shutil.move(f, os.path.join(docking_dir, 'inputs/' + f.split('/')[-1]))

print_str = f"Docking complete! Your results have been saved to: \
\n\t - docked ligands: {os.path.join(docking_directory, 'docked/')} \
\n\t - log files: {os.path.join(docking_directory, 'logs/')} \
\n\t - prepared input files: {os.path.join(docking_directory, 'inputs/')}"
        
print(print_str)

2019-08-13 08:16:11,215 - moleculekit.molecule - INFO - Removed 343 atoms. 2773 atoms remaining in the molecule.
2019-08-13 08:16:25,965 - moleculekit.tools.preparationdata - INFO - The following residues are in a non-standard state: LYS     5  A (LYN), HIS   103  A (HID), HIS   104  A (HID), HIS  1253  B (HID), HIS  1279  B (HID), HIS  1292  B (HID), HIS  1312  B (HIP), HIS  1341  B (HID), HIS  1353  B (HIE), HIS  1410  B (HID)
2019-08-13 08:16:25,968 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state: the pKa of 10 residues is within 1.0 units of pH 7.0.
2019-08-13 08:16:25,971 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    LYS     5  A (pKa= 6.51)
2019-08-13 08:16:25,973 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    HIS   103  A (pKa= 6.30)
2019-08-13 08:16:25,975 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    HIS  1253  B (pKa= 6.39)
2019-08-13 08:16:25,976 - molecul

IsADirectoryError: [Errno 21] Is a directory: '/data/docking/XX02KALRNA-x1389_1/in_ligs/'